In [1]:
import pandas as pd
import numpy as np
import os

# Выбросы данных

**Определение:** *Выброс* - это наблюдение, которое отклоняется настолько сильно от других наблюдений, что вызывает подозрения в том, что оно было сгенерировано другим механизмом.

**Примечание:** Выбросы, в зависимости от контекста, либо требуют особого внимания, либо должны быть полностью игнорированы. Например, необычная транзакция на кредитной карте обычно является признаком мошеннической деятельности, в то время как высота человека в 1600 см, скорее всего, обусловлена ошибкой измерения и должна быть отфильтрована или заменена на другое значение.

## Значение выбросов

Присутствие выбросов может:

- Сделать алгоритм неспособным нормально работать
- Внести шумы в набор данных
- Сделать выборки менее репрезентативными

Некоторые алгоритмы чрезвычайно чувствительны к выбросам. Например, **Adaboost** может рассматривать выбросы как "сложные" случаи и наделять их значительными весами, что может привести к модели с плохой обобщающей способностью. Любые алгоритмы, основанные на среднем и дисперсии, чувствительны к выбросам, так как эти статистические показатели сильно зависят от экстремальных значений.

С другой стороны, некоторые алгоритмы более устойчивы к выбросам. Например, **деревья решений** обычно игнорируют наличие выбросов при создании ветвей своих деревьев. Обычно деревья разделяют данные, задавая вопрос вида "переменная x >= значение t", и, следовательно, выброс попадет на обе стороны ветви, но будет обрабатываться так же, как и остальные значения, независимо от своей величины.


## Загрузка данных

In [2]:
use_cols = [
    'Pclass', 'Sex', 'Age', 'Fare', 'SibSp',
    'Survived'
]


data = pd.read_csv('./data/titanic.csv', usecols=use_cols)
data.head(3)
print(data.shape)

(891, 6)


In [3]:
pd.Series(data.Fare.unique()).sort_values()

104      0.0000
163      4.0125
245      5.0000
152      6.2375
240      6.4375
         ...   
164    227.5250
75     247.5208
148    262.3750
23     263.0000
127    512.3292
Length: 248, dtype: float64

## Поиск выбросов по заданным значениям

In [4]:
def outlier_detect_arbitrary(data, col, upper_fence, lower_fence):
    '''
    Функция для поиска выбросов в данных на основе заданных верхней и нижней границ.

    Parameters:
    data (DataFrame): Исходные данные.
    col (str): Название столбца, в котором нужно найти выбросы.
    upper_fence (float): Верхняя граница для определения выбросов.
    lower_fence (float): Нижняя граница для определения выбросов.

    Returns:
    pd.Series: Серия с булевыми значениями, указывающими на наличие выбросов.
    tuple: Кортеж, содержащий верхнюю и нижнюю границы, использованные для поиска выбросов.
    '''

    # Формируем кортеж с верхней и нижней границами для поиска выбросов.
    para = (upper_fence, lower_fence)

    # Создаем временный DataFrame, в котором для каждой строки указываем, превышает ли значение в столбце заданные границы.
    tmp = pd.concat([data[col] > upper_fence, data[col] < lower_fence], axis=1)

    # Находим индексы строк, содержащих хотя бы один выброс.
    outlier_index = tmp.any(axis=1)

    # Выводим информацию о найденных выбросах.
    print('Найдено выбросов:', outlier_index.value_counts()[1])
    print('Процент выбросов от всех данных:', outlier_index.value_counts()[1] / len(outlier_index))

    # Возвращаем серию с булевыми значениями и кортеж с границами.
    return outlier_index, para

In [5]:
index,para = outlier_detect_arbitrary(data=data,col='Fare',upper_fence=300,lower_fence=5)
print('Верхняя граница:',para[0],'\nНижняя граница:',para[1])

Найдено выбросов: 19
Процент выбросов от всех данных: 0.02132435465768799
Верхняя граница: 300 
Нижняя граница: 5


In [6]:
# Найдено 19 выбросов
data.loc[index,'Fare'].sort_values()

179      0.0000
806      0.0000
732      0.0000
674      0.0000
633      0.0000
597      0.0000
815      0.0000
466      0.0000
481      0.0000
302      0.0000
277      0.0000
271      0.0000
263      0.0000
413      0.0000
822      0.0000
378      4.0125
679    512.3292
737    512.3292
258    512.3292
Name: Fare, dtype: float64

## IQR метод

In [7]:
def outlier_detect_IQR(data, col, threshold=3):
    '''
    Поиск выбросов с использованием правила Interquartile Ranges, также известного как тест Тьюки.
    Рассчитывается межквартильный размах IQR = (75-й квантиль - 25-й квантиль).
    Верхняя граница = 75-й квантиль + (IQR * пороговое значение).
    Нижняя граница = 25-й квантиль - (IQR * пороговое значение).
    Данные за пределами этих границ считаются выбросами. Обычно используется пороговое значение 3.

    Parameters:
    data (DataFrame): Исходные данные.
    col (str): Название столбца, в котором нужно найти выбросы.
    threshold (int, optional): Пороговое значение для определения выбросов. По умолчанию равно 3.

    Returns:
    pd.Series: Серия с булевыми значениями, указывающими на наличие выбросов.
    tuple: Кортеж с верхней и нижней границами, используемыми для поиска выбросов.
    '''

    # Рассчитываем межквартильный размах (IQR).
    IQR = data[col].quantile(0.75) - data[col].quantile(0.25)

    # Вычисляем верхнюю и нижнюю границы для определения выбросов.
    Lower_fence = data[col].quantile(0.25) - (IQR * threshold)
    Upper_fence = data[col].quantile(0.75) + (IQR * threshold)

    # Формируем кортеж с использованными границами.
    para = (Upper_fence, Lower_fence)

    # Создаем временный DataFrame, в котором для каждой строки указываем, превышает ли значение в столбце заданные границы.
    tmp = pd.concat([data[col] > Upper_fence, data[col] < Lower_fence], axis=1)

    # Находим индексы строк, содержащих хотя бы один выброс.
    outlier_index = tmp.any(axis=1)

    # Выводим информацию о найденных выбросах.
    print('Найдено выбросов:', outlier_index.value_counts()[1])
    print('Процент выбросов от всех данных:', outlier_index.value_counts()[1] / len(outlier_index))

    # Возвращаем серию с булевыми значениями и кортеж с границами.
    return outlier_index, para


In [8]:
index,para = outlier_detect_IQR(data=data,col='Fare',threshold=5)
print('Верхняя граница:',para[0],'\nНижняя граница:',para[1])

Найдено выбросов: 31
Процент выбросов от всех данных: 0.03479236812570146
Верхняя граница: 146.448 
Нижняя граница: -107.53760000000001


In [9]:
# Найден 31 выброс
data.loc[index,'Fare'].sort_values()

31     146.5208
195    146.5208
305    151.5500
708    151.5500
297    151.5500
498    151.5500
609    153.4625
332    153.4625
268    153.4625
318    164.8667
856    164.8667
730    211.3375
779    211.3375
689    211.3375
377    211.5000
527    221.7792
700    227.5250
716    227.5250
557    227.5250
380    227.5250
299    247.5208
118    247.5208
311    262.3750
742    262.3750
341    263.0000
88     263.0000
438    263.0000
27     263.0000
679    512.3292
258    512.3292
737    512.3292
Name: Fare, dtype: float64

## Выделение выбросов через удаленность в N сигм

In [10]:
def outlier_detect_mean_std(data, col, threshold=3):
    '''
    Поиск выбросов с использованием метода удаленности от среднего в N сигм - mu +/- N*std. 
    Обычно пороговое значение threshold равно 3.
    
    Этот метод может быть неэффективным для поиска выбросов, так как выбросы могут увеличивать стандартное отклонение.
    Чем больше выброс, тем больше стандартное отклонение.
    
    Parameters:
    data (DataFrame): Исходные данные.
    col (str): Название столбца, в котором нужно найти выбросы.
    threshold (int, optional): Пороговое значение для определения выбросов. По умолчанию равно 3.

    Returns:
    pd.Series: Серия с булевыми значениями, указывающими на наличие выбросов.
    tuple: Кортеж с верхней и нижней границами, используемыми для поиска выбросов.
    '''

    # Вычисляем верхнюю и нижнюю границы для определения выбросов.
    Upper_fence = data[col].mean() + threshold * data[col].std()
    Lower_fence = data[col].mean() - threshold * data[col].std()

    # Формируем кортеж с использованными границами.
    para = (Upper_fence, Lower_fence)

    # Создаем временный DataFrame, в котором для каждой строки указываем, превышает ли значение в столбце заданные границы.
    tmp = pd.concat([data[col] > Upper_fence, data[col] < Lower_fence], axis=1)

    # Находим индексы строк, содержащих хотя бы один выброс.
    outlier_index = tmp.any(axis=1)

    # Выводим информацию о найденных выбросах.
    print('Найдено выбросов:', outlier_index.value_counts()[1])
    print('Процент выбросов от всех данных:', outlier_index.value_counts()[1] / len(outlier_index))

    # Возвращаем серию с булевыми значениями и кортеж с границами.
    return outlier_index, para


In [11]:
index,para = outlier_detect_mean_std(data=data,col='Fare',threshold=3)
print('Верхняя граница:',para[0],'\nНижняя граница:',para[1])

Найдено выбросов: 20
Процент выбросов от всех данных: 0.02244668911335578
Верхняя граница: 181.2844937601173 
Нижняя граница: -116.87607782296811


In [12]:
# найдено 20 выбросов
data.loc[index,'Fare'].sort_values()

779    211.3375
730    211.3375
689    211.3375
377    211.5000
527    221.7792
716    227.5250
700    227.5250
380    227.5250
557    227.5250
118    247.5208
299    247.5208
311    262.3750
742    262.3750
27     263.0000
341    263.0000
88     263.0000
438    263.0000
258    512.3292
737    512.3292
679    512.3292
Name: Fare, dtype: float64

## MAD метод

Поиск выбросов с помощью Median and Median Absolute Deviation Method (MAD)

In [13]:
def outlier_detect_MAD(data, col, threshold=3.5):
    """
    Обнаружение выбросов с использованием метода медианы и медианного абсолютного отклонения (MAD).
    Рассчитывается медиана значений в столбце. Затем рассчитывается разница между каждым значением
    и медианой этого столбца. Эти разницы выражаются в виде их абсолютных значений, а затем
    вычисляется новая медиана этой абсолютной разницы, умноженная на константу (которая равна 0.6745),
    что позволяет получить среднее абсолютное отклонение (MAD).
    Если значение находится на определенном расстоянии от медианы остатков,
    оно классифицируется как выброс. Порог по умолчанию — 3.5 MAD.
    
    Этот метод обычно более эффективен, чем метод среднего и стандартного отклонения для обнаружения выбросов.
    Однако он может быть слишком агрессивным в классификации значений, которые на самом деле не сильно различаются.
    Кроме того, если более 50% точек данных имеют одинаковое значение, MAD вычисляется как 0,
    поэтому любое значение, отличное от медианы, классифицируется как выброс.

    Parameters:
    data (DataFrame): Исходные данные.
    col (str): Название столбца, в котором нужно найти выбросы.
    threshold (float, optional): Пороговое значение для определения выбросов. По умолчанию равно 3.5.

    Returns:
    pd.Series: Серия с булевыми значениями, указывающими на наличие выбросов.
    """

    # Вычисляем медиану значений в столбце.
    median = data[col].median()

    # Рассчитываем медианное абсолютное отклонение (MAD).
    median_absolute_deviation = np.median([np.abs(y - median) for y in data[col]])

    # Рассчитываем модифицированные Z-оценки для каждого значения в столбце.
    modified_z_scores = pd.Series([0.6745 * (y - median) / median_absolute_deviation for y in data[col]])

    # Определяем индексы значений, которые считаются выбросами.
    outlier_index = np.abs(modified_z_scores) > threshold

    # Выводим информацию о найденных выбросах.
    print('Найдено выбросов:', outlier_index.value_counts()[1])
    print('Процент выбросов от всех данных:', outlier_index.value_counts()[1] / len(outlier_index))

    # Возвращаем серию с булевыми значениями, указывающими на наличие выбросов.
    return outlier_index


In [14]:
# наиболее жесткий способ поиск выбросов для нашего сета - 19%
index = outlier_detect_MAD(data=data,col='Fare',threshold=3)

Найдено выбросов: 171
Процент выбросов от всех данных: 0.1919191919191919


##  Заполнение выбросов случайным значением

In [15]:
# используем любой из представленных данных выше для детекции
index,para = outlier_detect_arbitrary(data=data,col='Fare',upper_fence=300,lower_fence=5)
print('Верхняя граница:',para[0],'\nНижняя граница:',para[1])

Найдено выбросов: 19
Процент выбросов от всех данных: 0.02132435465768799
Верхняя граница: 300 
Нижняя граница: 5


In [16]:
data[255:275]

,Survived,Pclass,Sex,Age,SibSp,Fare
255,1,3,female,29.0,0,15.2458
256,1,1,female,NaN,0,79.2000
257,1,1,female,30.0,0,86.5000
258,1,1,female,35.0,0,512.3292
259,1,2,female,50.0,0,26.0000
260,0,3,male,NaN,0,7.7500
261,1,3,male,3.0,4,31.3875
262,0,1,male,52.0,1,79.6500
263,0,1,male,40.0,0,0.0000
264,0,3,female,NaN,0,7.7500


In [17]:
def impute_outlier_with_arbitrary(data, outlier_index, value, col=[]):
    """
    Заполнение выбросов заданным произвольным значением.

    Parameters:
    data (DataFrame): Исходные данные.
    outlier_index (pd.Series): Серия с булевыми значениями, указывающими на наличие выбросов.
    value: Значение, которым нужно заменить выбросы.
    col (list, optional): Список столбцов, в которых нужно произвести замену выбросов. По умолчанию пустой список.

    Returns:
    pd.DataFrame: Копия исходных данных с произведенной заменой выбросов.
    """
    
    # Создаем копию исходных данных.
    data_copy = data.copy(deep=True)

    # Проходим по каждому столбцу из списка col и заменяем выбросы заданным значением.
    for i in col:
        data_copy.loc[outlier_index, i] = value

    # Возвращаем копию данных с произведенной заменой выбросов.
    return data_copy


In [18]:
# индексы 258,263,271 заменили
data2 = impute_outlier_with_arbitrary(data=data,outlier_index=index,
                                         value=-999,col=['Fare'])
data2[255:275]

,Survived,Pclass,Sex,Age,SibSp,Fare
255,1,3,female,29.0,0,15.2458
256,1,1,female,NaN,0,79.2000
257,1,1,female,30.0,0,86.5000
258,1,1,female,35.0,0,-999.0000
259,1,2,female,50.0,0,26.0000
260,0,3,male,NaN,0,7.7500
261,1,3,male,3.0,4,31.3875
262,0,1,male,52.0,1,79.6500
263,0,1,male,40.0,0,-999.0000
264,0,3,female,NaN,0,7.7500


## Windsorization

In [22]:
def windsorization(data, col, para, strategy='both'):
    """
    Применение метода виндзоризации к столбцу данных.

    Parameters:
    data (DataFrame): Исходные данные.
    col (str): Название столбца, который нужно подвергнуть виндзоризации.
    para (tuple): Кортеж, содержащий два значения (верхнее и нижнее ограничения) для виндзоризации.
    strategy (str, optional): Стратегия виндзоризации. Может быть 'both' (по умолчанию), 'top' (только верхнее ограничение) или 'bottom' (только нижнее ограничение).

    Returns:
    pd.DataFrame: Копия исходных данных с примененной виндзоризацией.
    """
    
    # Создаем копию исходных данных.
    data_copy = data.copy(deep=True)  
    
    # Применяем виндзоризацию в зависимости от выбранной стратегии.
    if strategy == 'both':
        data_copy.loc[data_copy[col] > para[0], col] = para[0]
        data_copy.loc[data_copy[col] < para[1], col] = para[1]
    elif strategy == 'top':
        data_copy.loc[data_copy[col] > para[0], col] = para[0]
    elif strategy == 'bottom':
        data_copy.loc[data_copy[col] < para[1], col] = para[1]
    
    # Возвращаем копию данных с примененной виндзоризацией.
    return data_copy


In [23]:
index,para = outlier_detect_arbitrary(data,'Fare', 300, 5)
print('Верхняя граница:',para[0],'\nНижняя граница:',para[1])

Найдено выбросов: 19
Процент выбросов от всех данных: 0.02132435465768799
Верхняя граница: 300 
Нижняя граница: 5


In [24]:
# индексы 258,263,271 заменили
data3 = windsorization(data=data,col='Fare',para=para,strategy='both')
data3[255:275]

,Survived,Pclass,Sex,Age,SibSp,Fare
255,1,3,female,29.0,0,15.2458
256,1,1,female,NaN,0,79.2000
257,1,1,female,30.0,0,86.5000
258,1,1,female,35.0,0,300.0000
259,1,2,female,50.0,0,26.0000
260,0,3,male,NaN,0,7.7500
261,1,3,male,3.0,4,31.3875
262,0,1,male,52.0,1,79.6500
263,0,1,male,40.0,0,5.0000
264,0,3,female,NaN,0,7.7500


## Удаление выбросов


In [25]:
index,para = outlier_detect_arbitrary(data,'Fare',300,5)
print('Верхняя граница:',para[0],'\nНижняя граница:',para[1])

Найдено выбросов: 19
Процент выбросов от всех данных: 0.02132435465768799
Верхняя граница: 300 
Нижняя граница: 5


In [26]:
# удаляем просто значения >300 или <5

data4 = data[~index]

print(data4.Fare.max())
print(data4.Fare.min())

263.0
5.0


## Заполнение медианой/модой/средним

In [27]:
# use any of the detection method above
index,para = outlier_detect_arbitrary(data,'Fare',300,5)
print('Верхняя граница:',para[0],'\nНижняя граница:',para[1])

Найдено выбросов: 19
Процент выбросов от всех данных: 0.02132435465768799
Верхняя граница: 300 
Нижняя граница: 5


In [28]:
def impute_outlier_with_avg(data, col, outlier_index, strategy='mean'):
    """
    Заполнение выбросов медианой, модой или средним значением столбца.

    Parameters:
    data (DataFrame): Исходные данные.
    col (str): Название столбца, в котором нужно произвести замену выбросов.
    outlier_index (pd.Series): Серия с булевыми значениями, указывающими на наличие выбросов.
    strategy (str, optional): Стратегия замены выбросов. Может быть 'mean' (по умолчанию), 'median' или 'mode'.

    Returns:
    pd.DataFrame: Копия исходных данных с произведенной заменой выбросов.
    """
    
    # Создаем копию исходных данных.
    data_copy = data.copy(deep=True)

    # В зависимости от выбранной стратегии, заменяем выбросы соответствующим значением (средним, медианой или модой).
    if strategy == 'mean':
        data_copy.loc[outlier_index, col] = data_copy[col].mean()
    elif strategy == 'median':
        data_copy.loc[outlier_index, col] = data_copy[col].median()
    elif strategy == 'mode':
        data_copy.loc[outlier_index, col] = data_copy[col].mode()[0]
    
    # Возвращаем копию данных с произведенной заменой выбросов.
    return data_copy


In [29]:
# индексы 258,263,271 заменили
data5 = impute_outlier_with_avg(data=data,col='Fare',
                                outlier_index=index,strategy='mean')
data5[255:275]

,Survived,Pclass,Sex,Age,SibSp,Fare
255,1,3,female,29.0,0,15.245800
256,1,1,female,NaN,0,79.200000
257,1,1,female,30.0,0,86.500000
258,1,1,female,35.0,0,32.204208
259,1,2,female,50.0,0,26.000000
260,0,3,male,NaN,0,7.750000
261,1,3,male,3.0,4,31.387500
262,0,1,male,52.0,1,79.650000
263,0,1,male,40.0,0,32.204208
264,0,3,female,NaN,0,7.750000
